In [32]:
import os
import numpy as np
import networkx as nx
import pandas as pd
from random import gauss
import time
from tqdm import tqdm

In [33]:
# Function to read a CSV file as a binary matrix
def read_csv_as_binary_matrix(file_path):
    df = pd.read_csv(file_path, header=None, delimiter=";", dtype=int)
    return df.values

# Define functions for calculating aggregated matrices
def calculate_aggregated_matrix_2x2(original_matrix):
    num_rows, num_cols = original_matrix.shape
    aggregated_matrix = np.zeros((num_rows // 2, num_cols // 2), dtype=float)

    for i in range(0, num_rows, 2):
        for j in range(0, num_cols, 2):
            square = original_matrix[i:i + 2, j:j + 2]
            average = square.mean()
            row_idx = i // 2
            col_idx = j // 2
            aggregated_matrix[row_idx, col_idx] = round(average, 2)

    return aggregated_matrix

def calculate_aggregated_matrix_3x3(original_matrix):
    num_rows, num_cols = original_matrix.shape
    aggregated_matrix = np.zeros((num_rows // 3, num_cols // 3), dtype=float)

    for i in range(0, num_rows, 3):
        for j in range(0, num_cols, 3):
            square = original_matrix[i:i + 3, j:j + 3]
            average = square.mean()
            row_idx = i // 3
            col_idx = j // 3
            aggregated_matrix[row_idx, col_idx] = round(average, 2)

    return aggregated_matrix

def calculate_aggregated_matrix_4x4(original_matrix):
    num_rows, num_cols = original_matrix.shape
    aggregated_matrix = np.zeros((num_rows // 4, num_cols // 4), dtype=float)

    for i in range(0, num_rows, 4):
        for j in range(0, num_cols, 4):
            square = original_matrix[i:i + 4, j:j + 4]
            average = square.mean()
            row_idx = i // 4
            col_idx = j // 4
            aggregated_matrix[row_idx, col_idx] = round(average, 2)

    return aggregated_matrix

def calculate_betas(aggregated_matrix, Sigma):
    element_num = aggregated_matrix.shape[0]
    interaction_coeffi1 = np.zeros((element_num, element_num))
    interaction_coeffi2 = np.zeros((element_num, element_num))

    for i in range(element_num):
       # Sigma1 = np.random.normal(0, 1)  # Random Sigma for coefficient 1
        interaction_coeffi1[0, i] = np.exp(-abs(gauss(aggregated_matrix[i,i], Sigma)))

        for j in range(element_num):
            if i != j:  # Avoid self-interaction
                #  Sigma2 = np.random.normal(0, 1)  # Random Sigma for coefficient 2
                interaction_coeffi2[i, j] = np.exp(-abs(gauss(aggregated_matrix[i,j], Sigma)))

    return interaction_coeffi1, interaction_coeffi2

def calculate_fitness(X, interaction_coeffi1, interaction_coeffi2, aggregated_matrix):
    element_num = aggregated_matrix.shape[0]
    fitness = 0.0

    for i in range(element_num):
        fitness += interaction_coeffi1[0, i] * X[i]
        for j in range(element_num):
            if i != j:
                fitness += interaction_coeffi2[i, j] * X[i] * X[j]

    return fitness

def generate_landscape(n, aggregated_matrix, interaction_coeffi1, interaction_coeffi2):
    fitness_vec = []
    fitness_pos = []

    for rr in range(pow(2,n)):
        # Convert rr to binary format and pad it to the desired length
        binary_vector = format(rr, f'0{n}b')
        bin_vec = binary_vector
        position_11 = int(bin_vec, 2)
        fitness_pos.append(position_11)
        binary_vector = ','.join(binary_vector)

        # Replace 0 with -1
        binary_vector = binary_vector.replace('0', '-1')

        # Convert the binary string to a list of integers
        X = [int(x) for x in binary_vector.split(",")]

        # Call calculate_fitness
        fitness = calculate_fitness(X, interaction_coeffi1, interaction_coeffi2, aggregated_matrix)
        fitness_vec.append(fitness)

        #print(f'Fitness for X={X}: {fitness}')
        #print("Interaction Coefficients (1):")
        #print(interaction_coeffi1)
        #print("\nInteraction Coefficients (2):")
        #print(interaction_coeffi2)

    return fitness_vec, fitness_pos

def local_optima(fitness_vector, fitness_pos, Sigma, landscape_id, file_name):
    local_optima_results = []
    count_local_optima = 0

    for i in range(len(fitness_vector)):
        binary_vector = format(fitness_pos[i], f'0{n}b')
        binary_vector = binary_vector.replace('-1', '0')
        position_11 = int(binary_vector, 2)
        index_position_11 = fitness_pos.index(position_11)
        fitness_local = fitness_vector[index_position_11]
        is_local_optima = True

        for j in range(len(binary_vector)):
            neighbor_binary_vector_11 = list(binary_vector)

            if neighbor_binary_vector_11[j-1] == '0':
                neighbor_binary_vector_11[j-1] = '1'
            else:
                neighbor_binary_vector_11[j-1] = '0'

            #print("binary_vector, neighbor_binary_vector",binary_vector, neighbor_binary_vector_11)

            neighbor_binary_vector = ''.join(neighbor_binary_vector_11)
            neighbor_binary_vector = neighbor_binary_vector.replace('-1', '0')
            position_11 = int(neighbor_binary_vector, 2)
            #print("position_11, fitness_pos",position_11, fitness_pos)
            index_position_11 = fitness_pos.index(position_11)
            #print("index_position_11",index_position_11)
            neighbor_fitness_local = fitness_vector[index_position_11]
            #print("neighbor_fitness_local, fitness_local",neighbor_fitness_local, fitness_local)

            if neighbor_fitness_local > fitness_local:
                is_local_optima = False


        if is_local_optima == True:
            count_local_optima += 1

        local_optima_results.append((landscape_id, Sigma, i + 1, count_local_optima, neighbor_fitness_local, fitness_local,file_name))

    local_optima_df = pd.DataFrame(local_optima_results,
                                   columns=['Landscape ID', 'Sigma', 'Count', 'Count Local Optima','Neighbor_fitness_local', 'Fitness_local','File Name'])


    return local_optima_df


In [34]:
padding_value = 0

landscape_count = 500

# Folder path where CSV files are located
folder_path = "/Users/ilaydadonmez/Desktop/R/DSM - CSV Files"

file_names = [
    "DSM-6", "DSM-48", "DSM-53", "DSM-57", "DSM-2", "DSM-3", "DSM-4", "DSM-14", "DSM-19", "DSM-20", "DSM-21", "DSM-22", "DSM-63",
    "DSM-24", "DSM-42", "DSM-62", "DSM-31", "DSM-40", "DSM-39", "DSM-27", "DSM-9", "DSM-8", "DSM-26", "DSM-59",  "DSM-38", "DSM-7",
    "DSM-33", "DSM-10", "DSM-60", "DSM-15", "DSM-11", "DSM-29", "DSM-13", "DSM-12", "DSM-47", "DSM-45", "DSM-41", "DSM-61", "DSM-28.2",
    "DSM-28.1", "random_0", "hierarc", "block", "local"]

# Create a list of Sigma values for the 5 landscapes
sigma_values = [5, 10, 15, 20, 25]


In [35]:
# Create an empty list to store results for all landscapes and Sigmas
detailed_results = []
all_results = []
dsm_info = []

for file_name in tqdm(file_names):
    file_path = os.path.join(folder_path, file_name + ".csv")
    binary_matrix = read_csv_as_binary_matrix(file_path) * 5
    """
    # Print the original matrix and its size
    print(f"\nOriginal DSM for {file_name}:")
    for row in binary_matrix:
        print(" ".join(map(str, row)))
    #print(f"Original Matrix Size: {binary_matrix.shape}")
    """
    # Check the size of the original matrix
    num_rows, num_cols = binary_matrix.shape

    if 18 <= num_rows <= 26 and 18 <= num_cols <= 26:
        method = "2x2"
        # Use 2x2 aggregation for matrix sizes between 18 and 26
        if num_rows % 2 != 0:
            # If not divisible by 2, add one row and one column filled with the specified padding value
            binary_matrix = np.pad(binary_matrix, [(0, 1), (0, 1)], constant_values=padding_value)
        aggregated_matrix = calculate_aggregated_matrix_2x2(binary_matrix)
    elif 27 <= num_rows <= 39 and 27 <= num_cols <= 39:
        method = "3x3"
        # Use 3x3 aggregation for matrix sizes between 27 and 39
        while num_rows % 3 != 0:
            # Add one row and one column filled with the specified padding value until it's divisible by 3
            binary_matrix = np.pad(binary_matrix, [(0, 1), (0, 1)], constant_values=padding_value)
            num_rows, num_cols = binary_matrix.shape
        aggregated_matrix = calculate_aggregated_matrix_3x3(binary_matrix)
    elif 40 <= num_rows <= 57 and 40 <= num_cols <= 57:
        method = "4x4"
        # Use 4x4 aggregation for matrix sizes between 40 and 57
        while num_rows % 4 != 0:
            # Add one row and one column filled with the specified padding value until it's divisible by 4
            binary_matrix = np.pad(binary_matrix, [(0, 1), (0, 1)], constant_values=padding_value)
            num_rows, num_cols = binary_matrix.shape
        aggregated_matrix = calculate_aggregated_matrix_4x4(binary_matrix)
    elif num_rows == 12:
        method = "None"
        aggregated_matrix = binary_matrix
    else:
        # Handle other cases or raise an error
        raise ValueError("Unsupported matrix size")
    
    # Create a graph from the aggregated matrix
    G = nx.Graph(aggregated_matrix)

    # Find communities using the Louvain method
    communities = list(nx.community.asyn_lpa_communities(G, weight='weight'))

    # Calculate modularity for the communities
    modularity = nx.community.quality.modularity(G, communities)
    dsm_info.append((file_name,modularity,num_rows, binary_matrix.shape[0] , aggregated_matrix.shape[0], method))

    for Sigma in sigma_values:
        landscapes_results = []  # To store results for 5 landscapes for each Sigma
        for landscape_id in range(landscape_count):
            [interaction_coeffi1, interaction_coeffi2] = calculate_betas(aggregated_matrix, Sigma)
            n = len(aggregated_matrix)
            fitness_vector, fitness_pos = generate_landscape(n, aggregated_matrix, interaction_coeffi1,
                                                             interaction_coeffi2)
            local_optima_results = local_optima(fitness_vector, fitness_pos, Sigma, landscape_id, file_name)
            landscapes_results.append(local_optima_results)

        # Concatenate results for landscapes into one dataframe
        landscapes_df = pd.concat(landscapes_results, ignore_index=True)
        # Get only the last row for each landscape_id
        landscapes_df = landscapes_df.groupby('Landscape ID').tail(1)
        #print(landscapes_df)
        # Append detailed information for this Sigma

        all_results.append({'Sigma': Sigma, 'Landscapes DataFrame': landscapes_df, 'ID': len(all_results) + 1})



100%|███████████████████████████████████████████████████████████████████████████| 44/44 [58:37:13<00:00, 4796.21s/it]


In [36]:

dsm_info_df = pd.DataFrame(dsm_info,columns=['File Name','Modularity', 'Original Size', 'Modified Size','Aggregated Size', 'Method'])
# Create a detailed summary dataframe
detailed_summary_df = pd.concat([detailed_result['Landscapes DataFrame'] for detailed_result in all_results], ignore_index=True)

detailed_summary_df = detailed_summary_df.drop(['Neighbor_fitness_local', 'Fitness_local', 'Landscape ID','Count'], axis = 1)

# Group by 'File Name' and 'Sigma' and calculate max and mean separately
max_values = detailed_summary_df.groupby(["File Name", "Sigma"])['Count Local Optima'].max().reset_index()
mean_values = detailed_summary_df.groupby(["File Name", "Sigma"])['Count Local Optima'].mean().reset_index()

# Merge the max and mean values
detailed_summary_df = pd.merge(max_values, mean_values, on=["File Name", "Sigma"], suffixes=('_Max', '_Mean'))

# Rename columns for clarity
detailed_summary_df.columns = ['File Name', 'Sigma', 'Max Count Local Optima', 'Mean Count Local Optima']

#print(detailed_summary_df)

# Save the detailed summary dataframe to a CSV file
detailed_summary_csv_file_path = '/Users/ilaydadonmez/Desktop/detailed_summary_results.csv'
detailed_summary_df.to_csv(detailed_summary_csv_file_path, index=False)

# Concatenate all landscape results for all Sigmas into one dataframe
all_results_pd = pd.concat([detailed_result['Landscapes DataFrame'] for detailed_result in all_results], ignore_index=True)

# Save all results to a single CSV file
all_results_csv_file_path = '/Users/ilaydadonmez/Desktop/all_results.csv'
all_results_pd.to_csv(all_results_csv_file_path, index=False)

dsm_info_csv_file_path = '/Users/ilaydadonmez/Desktop/dsm_info.csv'
dsm_info_df.to_csv(dsm_info_csv_file_path, index=False)


In [37]:
detailed_summary_df

,File Name,Sigma,Max Count Local Optima,Mean Count Local Optima
0,DSM-10,5,10,3.184
1,DSM-10,10,20,5.728
2,DSM-10,15,28,6.940
3,DSM-10,20,24,7.976
4,DSM-10,25,32,7.904
...,...,...,...,...
215,random_0,5,8,3.034
216,random_0,10,16,4.620
217,random_0,15,16,5.294
218,random_0,20,16,5.500
